In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mutual_info_score
import statsmodels.api as sm
from scipy.stats import fisher_exact, barnard_exact

from statsmodels.stats.multitest import fdrcorrection_twostage, fdrcorrection

In [2]:
path = '../../PARKINSONS/PDproj/celldata/datawithGFstatus/clean/'
filenames = [line.rstrip() for line in open(path +'filenames.txt')]
filenames

['cellsRPnegGFstatus.csv',
 'cellsRPposGFstatus.csv',
 'cellshilicnegGFstatus.csv',
 'cellshilicposGFstatus.csv',
 'cellslipidnegGFstatus.csv',
 'cellslipidposGFstatus.csv',
 'mediumRPnegGFstatus.csv',
 'mediumRPposGFstatus.csv',
 'mediumhilicnegGFstatus.csv',
 'mediumhilicposGFstatus.csv']

### Fisher's exact test

In [55]:
path = '../../PARKINSONS/PDproj/celldata/datawithGFstatus/clean/'
filenames = [line.rstrip() for line in open(path +'filenames.txt')]
path_out = '../../PARKINSONS/PDproj/cellresults/fisher_exact/twoGroups/'

In [56]:
# CONFIGURATION
conf=2
configs = ['code 32', 'below threshold', 'code 32 or below threshold']

In [57]:
# LOOP through all files
for fnum in range(10):
    fname = filenames[fnum]
    fname_out = fname.split('GF')[0] + '.csv'

    data = pd.read_csv(path + fname,sep=';', header=None)
    Area = data.iloc[3:,8:48].to_numpy(dtype=float)

    # Get a threshold for binarization with fixed principle
    th =  Area.flatten().mean() - Area.flatten().std()

    mass=data.iloc[3:,4].to_numpy(dtype=str)
    rtime=data.iloc[3:,5].to_numpy(dtype=str)
    masstime=np.asarray(["{}@{}".format(m,t) for m,t in zip(mass, rtime)])
    compound=data.iloc[3:,0].to_numpy(dtype=str)

    group_inds=[np.arange(i*10,(i+1)*10) for i in range(4)]
    NOISEMASK=data.iloc[3:,88:].to_numpy(dtype=int)==32
    tags = ['aSYN','comb.','IFNg','UT']

    if(conf==0):
        HAS_NOISE=NOISEMASK.sum(axis=1)>0
    else:
        HAS_NOISE=np.ones((NOISEMASK.shape[0],))

    if(conf==0):
        b_area = NOISEMASK
    elif(conf==1):
        b_area = Area < th*np.ones_like(NOISEMASK)
    elif(conf==2):
        b_area = (Area < th*np.ones_like(NOISEMASK)) | NOISEMASK

    print(fname)
    if(conf>0):
        print('Threshold:',th)
    print('Binary meaning: 1 signifies', configs[conf],'\n')

    n_tests = int(np.math.factorial(len(tags)-1)*HAS_NOISE.sum())
    pvalues=np.zeros((n_tests,), dtype=float)
    feature_inds=np.zeros((n_tests,), dtype=int)
    first_true=np.zeros((n_tests,), dtype=int)
    second_true=np.zeros((n_tests,), dtype=int)
    tests=np.empty((n_tests,), dtype=object)
    #prior_odds=np.zeros((n_tests,), dtype=float)

    test_number = 0
    for first in range(3):
        for second in range(first+1,4):

            group = np.array([np.repeat([tag],10) for tag in [tags[first], tags[second]]]).flatten()

            for feat in range(b_area.shape[0]):

                # For conf = 0, filter features without any noise
                if(not HAS_NOISE[feat]):
                    continue

                inds = np.stack((group_inds[first], group_inds[second])).flatten()
                binary = b_area[feat,inds].flatten()

                if(np.all(binary==0)):
                    tab = np.array([[10,0],[10,0]])
                elif(np.all(binary==1)):
                    tab = np.array([[0,10],[0,10]])
                else:
                    tab = pd.crosstab(group, binary).to_numpy(dtype=int)

                test = fisher_exact(tab)
                podds, p = test

                pvalues[test_number]=p
                #prior_odds[test_number]=podds
                feature_inds[test_number]=feat
                tests[test_number]=tags[first] + "--" + tags[second]
                first_true[test_number]=binary[:10].sum()
                second_true[test_number]=binary[10:].sum()
                test_number += 1    


    pBonf=np.minimum(1,pvalues*len(pvalues))
    correction = fdrcorrection(pvalues, alpha=0.05, method='n', is_sorted=False)

    result = pd.DataFrame(
                            {' compound':compound[feature_inds],
                            ' masstime':masstime[feature_inds],
                            ' test':tests,
                            ' p_orig':pvalues,
                            ' p_Bonf':pBonf,
                            ' p_FDR':correction[1],
                            #' prior_odds':prior_odds,
                            ' true_1st':first_true,
                            ' true_2nd':second_true}
    )

    # PRINT summaries
    print("Number of tests:",n_tests)
    print("With p uncorrected < 0.05:",np.sum(result[' p_orig']<0.05))
    print("With p Bonferroni < 0.05:",np.sum(result[' p_Bonf']<0.05))
    print("With p FDR < 0.05:",np.sum(result[' p_FDR']<=0.05))
    print('------------------------------')
    
    # SAVE the results
    result_significant = result.iloc[np.where(result[' p_orig']<0.05)[0],:].sort_values(by = ' p_FDR')
    result_significant.to_csv(path_out + fname_out, sep=';', index=None)

cellsRPnegGFstatus.csv
Threshold: 16.134531380756044
Binary meaning: 1 signifies code 32 or below threshold 

Number of tests: 2478
With p uncorrected < 0.05: 110
With p Bonferroni < 0.05: 14
With p FDR < 0.05: 14
------------------------------
cellsRPposGFstatus.csv
Threshold: 16.44882319764381
Binary meaning: 1 signifies code 32 or below threshold 

Number of tests: 6306
With p uncorrected < 0.05: 201
With p Bonferroni < 0.05: 0
With p FDR < 0.05: 45
------------------------------
cellshilicnegGFstatus.csv
Threshold: 15.668454933511695
Binary meaning: 1 signifies code 32 or below threshold 

Number of tests: 786
With p uncorrected < 0.05: 5
With p Bonferroni < 0.05: 0
With p FDR < 0.05: 0
------------------------------
cellshilicposGFstatus.csv
Threshold: 16.154332812893664
Binary meaning: 1 signifies code 32 or below threshold 

Number of tests: 3036
With p uncorrected < 0.05: 93
With p Bonferroni < 0.05: 18
With p FDR < 0.05: 18
------------------------------
cellslipidnegGFstatus.

In [380]:
fisher_exact(np.array([[3,7],[7,3]]))

(0.1836734693877551, 0.1788954079975752)

### Alternative approach: Test (aSYN + UT) vs (IFNg + comb)

In [52]:
path = '../../PARKINSONS/PDproj/celldata/datawithGFstatus/clean/'
filenames = [line.rstrip() for line in open(path +'filenames.txt')]
path_out = '../../PARKINSONS/PDproj/cellresults/fisher_exact/twoGroups/'

In [53]:
conf=2
configs = ['code 32', 'below threshold', 'code 32 or below threshold']

In [54]:
# LOOP through the files
for fnum in range(10):
    
    fname = filenames[fnum]
    fname_out = fname.split('GF')[0] + '.csv'
    
    data = pd.read_csv(path + fname,sep=';', header=None)
    Area = data.iloc[3:,8:48].to_numpy(dtype=float)

    # Get a threshold for binarization with fixed principle
    th =  Area.flatten().mean() - .5*Area.flatten().std()

    mass=data.iloc[3:,4].to_numpy(dtype=str)
    rtime=data.iloc[3:,5].to_numpy(dtype=str)
    masstime=np.asarray(["{}@{}".format(m,t) for m,t in zip(mass, rtime)])
    compound=data.iloc[3:,0].to_numpy(dtype=str)

    NOISEMASK=data.iloc[3:,88:].to_numpy(dtype=int)==32
    tags = ['aSYN+UT', 'comb+IFNg']
    group = np.concatenate([np.repeat(['aSYN+UT'],10),np.repeat(['IFNg+comb'],20),np.repeat(['aSYN+UT'],10)])

    if(conf==0):
        HAS_NOISE=NOISEMASK.sum(axis=1)>0
    else:
        HAS_NOISE=np.ones((NOISEMASK.shape[0],))


    if(conf==0):
        b_area = NOISEMASK
    elif(conf==1):
        b_area = Area < th*np.ones_like(NOISEMASK)
    elif(conf==2):
        b_area = (Area < th*np.ones_like(NOISEMASK)) | NOISEMASK

    print(fname)
    if(conf>0):
        print('Threshold:',th)
    print('Binary meaning: 1 signifies', configs[conf],'\n')

    n_tests = int(HAS_NOISE.sum())
    pvalues=np.zeros((n_tests,), dtype=float)
    feature_inds=np.zeros((n_tests,), dtype=int)
    first_true=np.zeros((n_tests,), dtype=int)
    second_true=np.zeros((n_tests,), dtype=int)
    tests=np.empty((n_tests,), dtype=object)
    #prior_odds=np.zeros((n_tests,), dtype=float)

    test_number = 0
    for feat in range(b_area.shape[0]):
        if(not HAS_NOISE[feat]):
            continue
        binary = b_area[feat,:].flatten()            

        if(np.all(binary==0)):
            tab = np.array([[20,0],[20,0]])
        elif(np.all(binary==1)):
            tab = np.array([[0,20],[0,20]])
        else:
            tab = pd.crosstab(group, binary).to_numpy(dtype=int)

        test = fisher_exact(tab)
        podds, p = test

        pvalues[test_number]=p
        #prior_odds[test_number]=podds
        feature_inds[test_number]=feat
        tests[test_number]=tags[0] + "--" + tags[1]
        first_true[test_number]=binary[:10].sum() + binary[30:].sum()
        second_true[test_number]=binary[10:30].sum()
        test_number += 1    


    pBonf=np.minimum(1,pvalues*len(pvalues))
    correction = fdrcorrection(pvalues, alpha=0.05, method='n', is_sorted=False)

    result = pd.DataFrame(
                            {' compound':compound[feature_inds],
                            ' masstime':masstime[feature_inds],
                            ' test':tests,
                            ' p_orig':pvalues,
                            ' p_Bonf':pBonf,
                            ' p_FDR':correction[1],
                            #' prior_odds':prior_odds,
                            ' true_1st':first_true,
                            ' true_2nd':second_true}
    )
    # PRINT summaries
    print("Number of tests:",n_tests)
    print("With p uncorrected < 0.05:",np.sum(result[' p_orig']<0.05))
    print("With p Bonferroni < 0.05:",np.sum(result[' p_Bonf']<0.05))
    print("With p FDR < 0.05:",np.sum(result[' p_FDR']<=0.05))
    print('------------------------------')
    
    # SAVE the results
    result_significant = result.iloc[np.where(result[' p_orig']<0.05)[0],:].sort_values(by = ' p_FDR')
    result_significant.to_csv(path_out + fname_out, sep=';', index=None)

cellsRPnegGFstatus.csv
Threshold: 17.263892130872435
Binary meaning: 1 signifies code 32 or below threshold 

Number of tests: 413
With p uncorrected < 0.05: 58
With p Bonferroni < 0.05: 15
With p FDR < 0.05: 30
------------------------------
cellsRPposGFstatus.csv
Threshold: 17.809606199974105
Binary meaning: 1 signifies code 32 or below threshold 

Number of tests: 1051
With p uncorrected < 0.05: 149
With p Bonferroni < 0.05: 30
With p FDR < 0.05: 36
------------------------------
cellshilicnegGFstatus.csv
Threshold: 17.14643300749763
Binary meaning: 1 signifies code 32 or below threshold 

Number of tests: 131
With p uncorrected < 0.05: 7
With p Bonferroni < 0.05: 3
With p FDR < 0.05: 3
------------------------------
cellshilicposGFstatus.csv
Threshold: 17.583341244631434
Binary meaning: 1 signifies code 32 or below threshold 

Number of tests: 506
With p uncorrected < 0.05: 49
With p Bonferroni < 0.05: 13
With p FDR < 0.05: 15
------------------------------
cellslipidnegGFstatus.cs

In [42]:
result_significant = result.iloc[np.where(result[' p_orig']<0.05)[0],:].sort_values(by = ' p_FDR')
#result_significant.to_csv(path_out + fname_out, sep=';', index=None)

In [44]:
result_significant.head(20)

,compound,masstime,test,p_orig,p_Bonf,p_FDR,true_1st,true_2nd
93,"[(2R,3S,4R,5R)-5-(6-Amino-9H-purin-9-yl)-3,4-d...",558.06513@0.598,aSYN+UT--comb+IFNg,1.450889e-11,5.992171e-09,1.977978e-08,20,0
257,D-(+)-Tryptophan,203.08233@2.933,aSYN+UT--comb+IFNg,1.450889e-11,5.992171e-09,1.977978e-08,0,20
375,nan,810.56687@13.361,aSYN+UT--comb+IFNg,3.046867e-10,1.258356e-07,2.076877e-07,1,20
181,"(19R)-25-Amino-22-hydroxy-22-oxido-17,21,23-tr...",674.5143@13.432,aSYN+UT--comb+IFNg,3.046867e-10,1.258356e-07,2.076877e-07,20,1
313,nan,524.33683@10.692,aSYN+UT--comb+IFNg,3.351553e-09,1.384192e-06,1.827651e-06,2,20
265,nan,675.12236@0.604,aSYN+UT--comb+IFNg,5.818065e-09,2.402861e-06,2.266197e-06,19,1
368,5-(Hydroxymethyl)cytidine,272.08941@0.608,aSYN+UT--comb+IFNg,5.818065e-09,2.402861e-06,2.266197e-06,19,1
379,nan,812.58314@13.723,aSYN+UT--comb+IFNg,2.569524e-08,1.061214e-05,8.757497e-06,0,17
341,nan,452.27943@10.377,aSYN+UT--comb+IFNg,1.541715e-07,6.367281e-05,4.670665e-05,20,4
211,nan,635.14342@0.611,aSYN+UT--comb+IFNg,4.055380e-07,1.674872e-04,1.105729e-04,17,1


In [146]:
np.sum(result[' p_FDR']<=0.05)

26

In [147]:
np.sum(result[' p_orig']<0.05)

61

In [148]:
np.sum(result[' p_Bonf']<0.05)

19